## Mama I'm a criminal

In [67]:
from io import open
import spacy
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu

warnings.filterwarnings("ignore")
%matplotlib inline
plt.switch_backend('agg')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [48]:
spacy.cli.download("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 8.3 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [49]:
spacy.cli.download("ja_core_news_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 8.6 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_md')


In [50]:
spacy.cli.download("nl_core_news_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 5.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_md')


In [51]:
MAX_LENGTH = 10
HIDDEN_SIZE = 256
TEACHER_FORCING_RATIO = 0.5
RANDOM_STATE = 39
DATA_DIR = 'data/eng-nld/%s-%s.txt'
SOS_token = 0
EOS_token = 1
JA = spacy.load('ja_core_news_md')
EN = spacy.load('en_core_web_md')
NL = spacy.load('nl_core_news_md')

In [52]:
def clean_text(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([.!?])", r" \1", sentence)
    sentence = re.sub(r"[^a-zA-Z.!?]+", r" ", sentence)
    return sentence

In [53]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
        
    def tokenizeSentence(self,sentence):
        if self.name == 'eng':
            sentence = clean_text(sentence)
            return [token.text for token in EN.tokenizer(sentence)]
        elif self.name == 'jpn':
            return [token.text for token in JA.tokenizer(sentence)]
        elif self.name == 'nld':
            sentence = clean_text(sentence)
            return [token.text for token in NL.tokenizer(sentence)]
        
    def addSentence(self, sentence):
        sentence = self.tokenizeSentence(sentence)
        for word in sentence:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [54]:
def readLangs(lang1,lang2,reverse=False):
    print("Reading lines...")
    
    lines = open(DATA_DIR % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    pairs = []
    for line in lines:
        lan1,lan2, _ = line.split('\t')
        lan1 = clean_text(lan1)
        pairs.append([lan1,lan2])
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        source = Lang(lang2)
        target = Lang(lang1)
    else:
        source = Lang(lang1)
        target = Lang(lang2)

    return source, target, pairs

In [55]:
def filterPair(source,target,pair):
    return len(source.tokenizeSentence(pair[0])) < MAX_LENGTH and \
        len(target.tokenizeSentence(pair[1])) < MAX_LENGTH

def filterPairs(source,target,sentence):
    return [pair for pair in sentence if filterPair(source,target,pair)]

In [56]:
def prepareData(lang1, lang2, reverse=False):
    source, target, pairs= readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(source,target,pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Train-test spliting...")
    train_pairs, test_pairs = train_test_split(pairs,shuffle=True,random_state=RANDOM_STATE)
    print("Train pairs length is %s"% len(train_pairs))
    print("Test pairs length is %s"% len(test_pairs))
    print("Counting words...")
    for pair in train_pairs:
        source.addSentence(pair[0])
        target.addSentence(pair[1])
    print("Counted words:")
    print(source.name, source.n_words)
    print(target.name, target.n_words)
    return source, target, train_pairs, test_pairs


source, target, train_pairs, test_pairs = prepareData('eng', 'nld', False)
print(random.choice(train_pairs))

Reading lines...
Read 75298 sentence pairs
Trimmed to 62240 sentence pairs
Train-test spliting...
Train pairs length is 46680
Test pairs length is 15560
Counting words...
Counted words:
eng 7948
nld 11134
['tom is talking nonsense .', 'Tom praat onzin.']


In [57]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [58]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [59]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in lang.tokenizeSentence(sentence)]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(source, pair[0])
    target_tensor = tensorFromSentence(target, pair[1])
    return (input_tensor, target_tensor)

In [60]:

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [61]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [62]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [63]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [64]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(source, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(target.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [65]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(train_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [66]:
encoder1 = EncoderRNN(source.n_words, HIDDEN_SIZE).to(device)
attn_decoder1 = AttnDecoderRNN(HIDDEN_SIZE, target.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

7m 40s (- 107m 24s) (5000 6%) 4.5205


KeyboardInterrupt: 

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "hi , have a nice day .")
plt.matshow(attentions.numpy())
plt.show()

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + source.tokenizeSentence(input_sentence) +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("I resign.")

evaluateAndShowAttention("Tom lied.")

evaluateAndShowAttention("Excuse me.")

evaluateAndShowAttention("Tom is shoveling snow outside.")

In [ ]:
def calculate_bleu(pairs, encoder, decoder):

  bleu_scores = []
  
  for pair in pairs:
    
    input_tensor = tensorFromSentence(source, pair[0]) 
    target_tensor = tensorFromSentence(target, pair[1])

    output_words, attentions = evaluate(encoder, decoder, input_tensor) 
    output_sentence = ' '.join(output_words)

    bleu = sentence_bleu([target_tensor], output_sentence)  
    bleu_scores.append(bleu)
  
  print('Average BLEU score:', (sum(bleu_scores) / len(bleu_scores))*100)


calculate_bleu(test_pairs, encoder1, attn_decoder1)